#Exploring Altruism in Social Networks

This model shows how altruistic behaviour emerges in social networks from an entirely selfish group. The analysis is conducted by simulating the ultimatum game on a Barabasi-Albert network with different degree of preferential attachment. While previous work studied the emergence of altruistic behaviour in a well mixed population, this paper will study how social networks can limit the arise of altruism. Agents in the model are empathic and memoryless and the strategic update rule is natural selection. 

The work is published here https://www.researchgate.net/publication/269400895_Exploring_Altruism_in_Social_Networks

The model assumes that all agents $N$ are memoryless and empathic. each agent has a threshold $t_i$ which is an integer number indicates the amount of money the agent is willing to accept in the game, where $1 \le t_i \le M$. $M$ is total amount available each game.

In [1]:
def Initialization(N): 
    agentsdummy = np.zeros((2,N),dtype=int) #threshold, capital
    agentsdummy[0,:1000:1]= 1 
    agents = agentsdummy.T
    return agents

Network construction Barabsi-Alberts.

In [2]:
def NetworkConstruction(N,agents,degree):
    network = []
    G = nx.barabasi_albert_graph(N,degree)
    for i in xrange(N): # to group nodes to two groups selfish = 1, altruist =2
        if agents[i][0] > 1:
            G.add_node(i, Group=2)
        else:
             G.add_node(i, Group=1) 
    network = nx.to_dict_of_lists(G) #will use it to adjacent nodes later 
    return network,G

To get connected nodes 

In [3]:
def neighborhood(agent,network): 
    availablePlayers = network[agent]
    return availablePlayers

To choose players. In the previous model any player can be proposer and play with any other player as responder. Here any player can be chosen as proposer but only players in the proposer neighbourhood can be chosen as a responder.

In [4]:
def ChoosePlayers(agents, N, network): #choose the players, two agents each game like paper 1
    currProposer = randint(1, N) 
    availablePlayers = neighborhood(currProposer,network) # who she can play with
    currResponder = choice(availablePlayers)
    return currProposer,currResponder

Since players are empathic then the proposed amount will be equal to the proposer threshold $t_p$, so if the proposer threshold $t_p$ greater than the responder $t_r$ the offer will be accepted. Both players fitnesses will be updated as follow:

$$\eta_p = M - t_p$$
$$\eta_r = t_p$$

In [5]:
def TheGame(agents, currProposer, currResponder, M): #capital update following the game rules
    if agents[currProposer][0] >= agents[currResponder][0]: #assuming empathetic agents, where my threshold = proposal amount
        agents[currProposer][1] += (M - agents[currProposer][0])
        agents[currResponder][1] += agents[currProposer][0]
    return agents

After $S$ games, for each agent the agent with the maximum fitness in the neighborhood will substitute the agent with minimum fitness. 

In [6]:
def Selection(agents): 
    agentsdummy = agents.T
    possiblereplicaAgent = np.where(agentsdummy[1] == (max(agentsdummy[1])))
    replicaAgentIndex = choice(possiblereplicaAgent[0])
    possibleremovedAgent = np.where(agentsdummy[1] == (min(agentsdummy[1])))
    removedAgentIndex = choice(possibleremovedAgent[0])
    agents[removedAgentIndex][0] = agents[replicaAgentIndex][0]
    agents[removedAgentIndex][1] = agents[replicaAgentIndex][1]
    return agents, replicaAgentIndex

With probability $1/3$ the duplicated agent acceptance threshold will change $\pm 1$.

In [7]:
def Mutation(agents,replicaAgentIndex,M):
    mutationProb = random()
    if mutationProb > 0.666666667  and agents[replicaAgentIndex][0] < M:
        agents[replicaAgentIndex][0]= agents[replicaAgentIndex][0]+1
    if mutationProb < 0.333333333 and agents[replicaAgentIndex][0] > 1:
        agents[replicaAgentIndex][0]= agents[replicaAgentIndex][0]-1
       
    return agents

In [8]:
def Run(N,M, s, numOfGames,degree):
    clock = 1
    agents = Initialization(N)
    network,G = NetworkConstruction(N,agents,degree)

    while(clock != numOfGames):
        currProposer,currResponder = ChoosePlayers(agents, N, network)
        agents = TheGame(agents, currProposer, currResponder, M)
        if np.fmod(clock,s) == 0:
            agents,replicaAgentIndex = Selection(agents)
            agents = Mutation(agents,replicaAgentIndex,M)
        clock = clock+1
    frequency =  Counter(agents.T[0])
    averageThreshold = sum(agents.T[0])/N
    return averageThreshold, frequency

The model here called only once (to show resultes fast) in order to get accurate stastistical analysis simulations runs were 50. 

In [9]:
import numpy as np
import pylab as pl
import networkx as nx
from random import *
from collections import Counter
import pylab as plt

i = 0
averageThresholdAcrossRuns = []
FreqsAcrossRuns =[]
N = 1000
degree = 10
M = 100
s = 10000
numOfGames = 1
averageThreshold, frequency = Run(N, M, s, numOfGames,degree)